# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

It is clearly a classification problem, because we need a clear discrete answer: yes or no.

Classification can answer to a question: do wee need to intervent - yes or no, while regression gives us a continuous answer, which clearly isn't required here.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# Calculate number of students
n_students = len(student_data.index)

# Calculate number of features (minus 1, because "passed" is not a feature, but a target column)
n_features = len(student_data.columns) - 1

# Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# Calculate graduation rate
grad_rate = (n_passed / float(n_students)) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# Import Cross Validation functionality to perform splitting the data
from sklearn import cross_validation

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Calculate split percentage
splitPercentage = float(num_test) / (num_train + num_test)

print "Split percentage: {0:.2f}% ".format(splitPercentage * 100)

# Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_all, y_all, test_size = splitPercentage, stratify = y_all, random_state = 0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Split percentage: 24.05% 
Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

As we need to get a binary answer, it is better to choose among methods which are used for binary classification more often. I've chosen three learning models: K Nearest Neighbors, Logistic Regression and Support Vector Machine.

**K Nearest Neighbors is one the simplest ML algorithms:**

Real-World Applications:
- Optical Character Recognition.
- One of the most popular algorithms for text categorization or text mining.
- Often used for simulating daily precipitations and other weather variables.
- Stock market forecasting.

Advantages:
- The cost of the learning process is zero.
- No assumptions about the characteristics of the concepts to learn have to be done.
- Complex concepts can be learned by local approximation using simple procedures.

Disadvantages:
- The model can not be interpreted (there is no description of the learned concepts).
- It is computationally expensive to find the k nearest neighbours when the dataset is very large.
- Performance depends on the number of dimensions that we have (curse of dimensionality).

**Logistic Regression: **

Real-World Applications:
- It is used widely in many fields, which includes predicting of passing exams, voting preferences, consumer likelihod to purchase a product.

Advantages:
- It is very efficient in terms of time and memory requirement.
- It is not heavily affected by noise in the data. 

Disadvantages:
- If we have lots of features and a small dataset - doesn't perform well.
- It has problems with dealing with categorical features. 

**Support Vector Machine: **

Real-World Applications:
- Text (and hypertext) categorization.
- Image classification.
- Bioinformatics (Protein classification, Cancer classification).
- Hand-written character recognition.

Advantages:
- It is a great algorithm for our dataset as it relies heavily on boundary cases to build the separating curve. Which means that it can handle missing data for some "obvious" cases.
- It can efficiently handle large feature space.
- Do not rely on entire data.

Disadvantages:
- Not really our case, but it is not very efficient with a large number of observations.
- It is hard to find appropriate kernel sometitems. 

** Reasons to choose these algorithms: **
1. KNN: uses the least amount of computational cost, as we have a small dataset and a requirement to save money.
2. Logistic Regression: often used to perform exactly the same type of classification we need in our case. Also very efficient in time and memory.
3. SMV: it is great on a small dataset, because requires only boundary cases and can handle big feature space efficiently (our case).

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [21]:
# Import the three supervised learning models from sklearn
# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Support Vector Machine
from sklearn.svm import SVC

# KNN classifier
from sklearn.neighbors import KNeighborsClassifier

# Random State for each model consistent with splitting Random State = 0
randState = 0

# Initialize the three models
clf_A = LogisticRegression(random_state = randState)

clf_B = SVC(random_state = randState)

clf_C = KNeighborsClassifier()

classifiers = [clf_A, clf_B, clf_C]

# Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

trainingData = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]

# Execute the 'train_predict' function for each classifier and each training set size
for each in range(len(classifiers)):
    print classifiers[each]
    print "-------------------"
    for data in trainingData:
        train_predict(classifiers[each], data[0], data[1], X_test, y_test)
        print 


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
-------------------
Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0077 seconds
Made predictions in 0.0023 seconds.
F1 score for training set: 0.8759.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7500.

Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0041 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8532.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.8085.

Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0050 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8402.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7770.

S

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0077            |        0.0012          |       0.8759     |     0.7500      |
| 200               |       0.0041            |        0.0005          |       0.8532     |     0.8085      |
| 300               |       0.0050            |        0.0002          |       0.8402     |     0.7770      |

** Classifer 2 - Support Vector Machine**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0016              |         0.0008         |       0.8919     |    0.7737       |
| 200               |     0.0036              |         0.0019         |       0.8795     |    0.8182       |
| 300               |     0.0080              |         0.0029         |       0.8664     |    0.8289       |

** Classifer 3 - K Nearest Neighbors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0019             |         0.0024         |       0.8592     |     0.7910      |
| 200               |      0.0013             |         0.0016         |       0.8425     |     0.7941      |
| 300               |      0.0011             |         0.0039         |       0.8688     |     0.7714      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

After the data stratification we immediately get better results from the SVM, but it takes much longer to predict.

The best model in terms of cost and efficiency for this data is probably a Logistic Regression. Scores are consistent and the classificator works blazingly fast, while the difference in scores with the SVM is not huge.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

Logistic Regression takes the data and tries to calculate the probability of occurence of an event by fitting data on a logistic curve. It takes available information about previous students and if they passed or not, and then tries to calculate the probability of another student to pass the exams given the data.

**Training:**

As an example, if we wanted to predict success of students in their life, based on their scores, we would develop something called a model using training data.

We have scores (independent variable), we also know whether a person succeeded or not (dependent variable). We come up with some predictions and then look at how close our predictions were with the recorded data.

If we predicted 0.9 on someone (who succeded in his\her life), and in the same manner we are pretty close in all our predictions then we have a very developed and a good model. If we missed the correct answer, we would go about looking at various models (something called minimizing the squared error: we take a look at sum of errors for each prediction in each model and try to see if we can go even lower than it) and find out the model which fits very closely with our recorded data.

**Predicting:**

Then we can plug other people data scores into this model and it returns a number between 0 and 1. By looking at it, if it is greater than 0.5 -> a person is successful, or if it is lower than 0.5 -> person is unsuccessful. 


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [41]:
# Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

def gridSearch(clf, parameters):
    # Make an f1 scoring function using 'make_scorer' 
    f1_scorer = make_scorer(f1_score, pos_label="yes")

    # Perform grid search on the classifier using the f1_scorer as the scoring method
    grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer) 

    # Fit the grid search object to the training data and find the optimal parameters
    grid_obj = grid_obj.fit(X_train, y_train)

    # Get the estimator
    clf = grid_obj.best_estimator_

    return clf

# Create the parameters list you wish to tune
svcParameters = [
    {'C': [1, 10, 100], 'kernel': ['rbf'], 'gamma': ['auto']},
    {'C': [1, 10, 100, 1000], 'kernel': ['linear', 'rbf', 'poly']}
]

knnParams = {'n_neighbors': [2, 3, 4, 5],
            'weights': ['uniform', 'distance']}

regresParams = {'C': [0.5, 1.0, 10.0, 100.0],
               'max_iter': [100, 1000],
               'solver': ['sag', 'liblinear']}

randState = 0

classifiers = [gridSearch(SVC(random_state = randState), svcParameters), 
               gridSearch(KNeighborsClassifier(), knnParams),
               gridSearch(LogisticRegression(random_state = randState), regresParams)]

# Report the final F1 score for training and testing after parameter tuning
# I've tested all three of them just out of curiosity, I've chosen Logistic Regression initially.
for clf in classifiers:
    print clf
    print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
    print "-----------------"

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0049 seconds.
Tuned model has a training F1 score of 0.8664.
Made predictions in 0.0021 seconds.
Tuned model has a testing F1 score of 0.8289.
-----------------
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Made predictions in 0.0055 seconds.
Tuned model has a training F1 score of 0.8688.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.7714.
-----------------
LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='sag', tol=0.0001,
          verbose=0, warm_start=Fa

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

Tuned Logistic Regression is much faster than SVM, better than KNN in score and has an improvement of 2% in score after tuning: 
- Tuned model has a training F1 score of 0.8225.
- Tuned model has a testing F1 score of 0.7917.



> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.